In [45]:
import tensorflow as tf
import numpy as np
import os
import _Data
import _Model 
import time
from _utils import u_constant
path = u_constant.PATH_ROOT + "for learn/Python/RNN-for-Joint-NLU/"
train_data_path = path + "dataset/atis-2.train.w-intent.iob"
test_data_path = path + "dataset/atis-2.dev.w-intent.iob"
model_path = path + "model/"

In [50]:
imp.reload(_Model)
imp.reload(_Data)

<module '_Data' from 'C:\\Users\\Cigar\\Documents\\jupyter\\NLP_learn\\RNN_for_Joint_NLU\\_Data.py'>

In [51]:
data_info = _Data.DataUtils(train_data_path, test_data_path, length=50)

AttributeError: 'DataUtils' object has no attribute 're'

In [37]:
input_length = data_info.length  # 50
embedding_size = 64
hidden_size = 100
n_layers = 2
batch_size = 16
vocab_size = len(data_info.word2index)  # 871
slot_size = len(data_info.tag2index)  # 124
intent_size = len(data_info.intent2index)  # 22
epoch_num = 60

In [38]:
indexed_train_data  = data_info.to_index(data_info.train_data)  # 4478
indexed_test_data = data_info.to_index(data_info.test_data)  # 500

In [28]:
model = _Model.Model(input_length, embedding_size, hidden_size, vocab_size, 
                     slot_size, intent_size, batch_size)

In [ ]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 程序最多只能占用指定gpu80%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存

In [ ]:
def get_iternum_from_ckpt(ckpt_path):
    iters = ckpt_path.split(".ckpt-")[1]
    return int(iters)

In [ ]:
with tf.Session(graph=model.graph, config=config) as sess:
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt is not None:
        ckpt_path = ckpt.model_checkpoint_path
        model.saver.restore(sess, ckpt_path)
        iteration = get_iternum_from_ckpt(ckpt_path)
    else:
        sess.run(tf.global_variables_initializer())
        iteration = 1
    start = time.time()
    avg_loss = 0.0
    for epoch in range(epoch_num):
        for batch in data_info.get_batch(batch_size):
            # 执行一个 batch 的训练
            _, loss, decoder_prediction, intent, mask, slot_w = model.step(sess, "train", batch)
            avg_loss += loss
            if iteration % 100 == 0:
                end = time.time()
                print("Iteration: %d " % iteration, 
                      "Avg. Train_loss: %.4f" % (avg_loss / 100),
                      "%.4f sec / batch" % ((end - start) / 100))
                start = time.time()
                avg_loss = 0.0
            iteration += 1
        # 每个 epoch保存一次模型, 并测试
        model.saver.save(sess, model_path + "/seq2seq.ckpt", global_step=iteration)
        pred_slots = []
        slot_accs = []
        intent_accs = []
        decoder_prediction, intent = model.step(sess, "test", indexed_test_data)
        decoder_prediction = np.transpose(decoder_prediction, [1, 0])
        # 随机选出一个样本打印
        index = random.choice(range(len(indexed_test_data)))
        _sin, _true_length, _sout, _intent = indexed_test_data[index]
        print("Input Sentence   :",\
              data_info.recover_indexed_seq(_sin, "word")[:_true_length])
        print("Slot Truth       :",\
              data_info.recover_indexed_seq(_sout, "slot")[:_true_length])
        print("Slot Pred        :",\
              data_info.recover_indexed_seq(decoder_prediction[index], "slot")[:_true_length])
        print("Intent Truth     :", data_info.index2intent(_intent))
        print("Intent Pred      :", data_info.index2intent(intent[index]))
        
        
        
        
        
    